In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024 * 4)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:
#loading the data from the csv file into a pandas dataframe
df = pd.read_csv("chardata.csv")
df.shape

In [ ]:
df.rename(columns={'0':'label'}, inplace= True)

In [ ]:
#Splitting the dataset into target vector and label array
X = df.drop("label", axis= 1).values
y = df["label"].values

print(f"Shape of X and y: {X.shape, y.shape}")

In [ ]:
#this is a test cell. Not a part of the actual code
#see: https://stackoverflow.com/questions/66048202/image-resize-in-tensorflow
x_resize = tf.image.resize(tf.expand_dims(X[0].reshape(28,28), axis=-1), [64,64]).numpy()

plt.imshow(x_resize)

In [ ]:
#printing 5 random images to gain info about the dataset
for i in range(5):
    k = np.random.randint(low= 0,high= 372450)
    plt.imshow(X[k,:].reshape(28,28), cmap="gray")
    plt.title(y[k])
    plt.show()

In [ ]:
#creating the NN model
model = Sequential(
    [
        tf.keras.Input(shape=(784,),),
        Dense(200, activation= "relu"),
        Dense(150, activation= "relu"),
        Dense(100, activation= "relu"),
        Dense(50, activation= "relu"),
        Dense(26, activation= "linear")
    ]
)

model.summary()

In [ ]:
#splitting the dataset into training set and test set 

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 0.95)

print(f"Shape of X_test and X_train: {X_test.shape, X_train.shape}")

In [ ]:
#splitting the test set into test set and cross validation set

X_test, X_cv, y_test, y_cv = train_test_split(X_test, y_test, train_size= 0.4)

print(f"Shape of X_test and X_cv: {X_test.shape, X_cv.shape}")

In [ ]:
#compiling the model and fitting it to the training set

model.compile(
    loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),
    optimizer= tf.keras.optimizers.Adam(0.0003),
    jit_compile=True
)

model.summary()

In [ ]:
model.fit(X_train, y_train, epochs= 10)

In [ ]:
pred = model.predict(X_cv)

c = 0.
m = X_cv.shape[0]

for i in range(m):
    if(np.argmax(pred[i])!= y_cv[i]):
        c += 1


In [ ]:
print(f"Model accuaracy: {100- (c/m)*100}")

In [ ]:
model.save("CharRecognition.h5")